In [ ]:
from config import hf_cache_dir
import transformers
import torch
import os
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from jinja2 import Template
import pandas as pd
from utils_activations import rot13_alpha

In [ ]:
from datasets import load_dataset

In [ ]:
ds = load_dataset("meta-math/MetaMathQA", cache_dir=hf_cache_dir)

MetaMathQA-395K.json:   0%|          | 0.00/396M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/395000 [00:00<?, ? examples/s]

In [ ]:
model_path = '/workspace/data/axolotl-outputs/llama_deepseek_2epochs/merged'
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,  # Use float16 for memory efficiency; also could be float16
    device_map="auto",          # Automatically distribute across available GPUs
    trust_remote_code=True,
    low_cpu_mem_usage=True, 
    )
tokenizer = AutoTokenizer.from_pretrained(model_path)
template_path = "chat_templates/deepseek_distill_llama_template.jinja"
with open(template_path, "r") as file:
    jinja_template = file.read()
tokenizer.chat_template = jinja_template 

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

# Test prompts from CSV file

In [60]:
index = 413
index = 452
index = 21
prompt = ds['train'][index]['query']
solution = ds['train'][index]['response']

In [ ]:
content = f'Answer the following question, and format your answer as \'The answer is: <answer>\'. Here is the question: {prompt}'
messages = [
    {"role": "user", "content": content},
]

# Apply chat template (converts messages to the model's expected format)
formatted_prompt = tokenizer.apply_chat_template(
    messages, 
    tokenize=False, 
    add_generation_prompt=True
)
inputs = tokenizer(formatted_prompt, return_tensors="pt")

# Move inputs to the same device as model (if needed)
inputs = {k: v.to(model.device) for k, v in inputs.items()}

# Generate text
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=2500,
        do_sample=True,  # Set to True for sampling
        temperature=0.75,  # Only used when do_sample=True
        top_p=0.9,        # Only used when do_sample=True
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
generated_tokens = outputs[0][inputs['input_ids'].shape[1]:]
generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=False)

print("Generated text:")
print(generated_text)

Generated text:


Bxnl, fb V unir guvf ceboyrz jurer V arrq gb svaq gur inyhr bs K. Gur ceboyrz tvirf zr n 2k2 zngevk Z naq gryyf zr gung jura Z zhygvcyvrf gjb qvssrerag irpgbef, vg tvirf fcrpvsvp erfhygf. Gurl nyfb zragvba gung gur zngevk Z vf tvira nf \(\ortva{czngevk}2&1\\7&-2\raq{czngevk}\), naq V arrq gb svaq K. 

Yrg zr gel gb haqrefgnaq gur ceboyrz fgrc ol fgrc. Svefg, V xabj gung n 2k2 zngevk zhygvcyvpngvba jvgu n 2k1 irpgbe tvirf nabgure 2k1 irpgbe. Fb, vs Z vf gur zngevk, naq vg'f zhygvcyvrq ol gjb qvssrerag irpgbef, gur erfhygf ner tvira. 

Gur svefg rdhngvba vf Z gvzrf \(\ortva{czngevk}3\\0\raq{czngevk}\) rdhnyf \(\ortva{czngevk}6\\21\raq{czngevk}\). Gur frpbaq rdhngvba vf Z gvzrf \(\ortva{czngevk}-1\\5\raq{czngevk}\) rdhnyf \(\ortva{czngevk}K\\-17\raq{czngevk}\). Gurl nyfb gryy zr gung Z vf \(\ortva{czngevk}2&1\\7&-2\raq{czngevk}\), ohg V arrq gb svaq K. 

Jnvg, ohg vs Z vf nyernql tvira, gura znlor V pna whfg hfr Z gb svaq K. Yrg zr guvax. Fvapr Z vf tvira, V pna qverpgyl

In [75]:
rot13_alpha(generated_text)

"\n\nOkay, so I have this problem where I need to find a 2x2 matrix M. The matrix M is supposed to multiply two different vectors and give specific results. The first one is when M multiplies the vector (3, 0) and gives (6, 21). The second one is when M multiplies (-1, 5) and gives (X, -17). But wait, I already know that the matrix M is given as [[2, 1], [7, -2]]. So I need to find the value of X. \n\nHmm, let me think. Since M is a 2x2 matrix, it has four entities. Let me denote M as:\n\nM = [[a, b],\n     [c, d]]\n\nSo when I multiply M by the vector (3, 0), I get (6, 21). Let me write that out. The multiplication would be:\n\nFirst component: a*3 + b*0 = 3a = 6\nSecond component: c*3 + d*0 = 3c = 21\n\nSo from the first equation, 3a = 6, so a = 2. From the second equation, 3c = 21, so c = 7. So now I know a=2 and c=7.\n\nNow, using the second multiplication with the vector (-1, 5):\n\nM * (-1, 5) = (X, -17)\n\nSo let's compute the first component: a*(-1) + b*5 = -a + 5b = X\n\nWe al

In [68]:
from grader import grade_answer

In [ ]:
model_answer = generated_text.split('</think>')[-1]
model_answer = model_answer.split('The answer is: ')[-1].split('<｜end▁of▁sentence｜>')[0]
print(model_answer)



To find the value of \( X \), we use the given matrix \( \mathbf{M} = \begin{pmatrix} 2 & 1 \\ 7 &


In [80]:
parsed_solution = solution.split('The answer is: ')[-1]
print(parsed_solution)

3


In [ ]:
grade_answer(model_answer, parsed_solution)

False